In [ ]:
project_path = "/home/jupyter"
import os
import sys
sys.path.append(project_path)

from google.cloud import bigquery
import importlib
import plotly.express as px

import numpy as np
import pandas as pd
from datetime import datetime

import ft_digital_trade.src.utils.read_data as read_utils
import ft_digital_trade.src.utils.clean_utils as clean_utils
import ft_digital_trade.src.utils.calculation_utils as calc_utils
import ft_digital_trade.src.utils.plot_utils as plot_utils

client = bigquery.Client()

visa_data = read_utils.read_visa(cardholder_origin = "uk", cardholders_location = "uk", spend_location = "uk")

visa = calc_utils.calculate_visa(visa_data)
visa = clean_utils.rename_columns(df = visa, suffix = '_spoc')

global_cards = read_utils.read_global_cards()
global_cards = clean_utils.clean_global(global_cards)
global_cards = calc_utils.calculate_global(global_cards, 'card')

global_spend = read_utils.read_global_spend()
global_spend = clean_utils.clean_global(global_spend)
global_spend = calc_utils.calculate_global(global_spend, 'spend')

global_df = global_cards.merge(global_spend, how = 'inner', on = 'year', suffixes = ('_cards', '_spend'))
global_df = clean_utils.rename_columns(df = global_df, suffix = '_global')

uk_finance = read_utils.read_uk_finance()
uk_finance = clean_utils.clean_uk_finance(uk_finance)
uk_finance = calc_utils.calculate_uk_finance(uk_finance)
uk_finance = uk_finance[['year', 'cardholders','total value of purchases',"total volume of purchases"]]
uk_finance = clean_utils.rename_columns(df = uk_finance , suffix = '_uk_finance')

boe = read_utils.read_boe()
boe = clean_utils.clean_boe(boe)
boe = calc_utils.calculate_boe(boe)
boe = clean_utils.rename_columns(df = boe , suffix = '_boe')

In [ ]:
link = read_utils.read_link()

merged = visa.merge(uk_finance, how = 'outer', on = 'year')
merged = merged.merge(boe, how = 'outer', on = 'year')
merged = merged.merge(global_df, how = 'outer', on = 'year')

cardholders = merged[['year','cardholders_spoc','cardholders_uk_finance','visa_total_cards_global','total_cards_global', 'visa_marketshare_cards_global']]
cardholders = cardholders.copy()
cardholders['uk_finance_marketshare'] = cardholders['cardholders_spoc'] / cardholders['cardholders_uk_finance'] *100
cardholders['global_marketshare'] = cardholders['cardholders_spoc'] / cardholders['total_cards_global'] *100
#melt df for charts
cardholders = pd.melt(cardholders, id_vars='year',var_name='Data source', value_name='value')
cardholders = calc_utils.calculate_index(df = cardholders)

spend = merged[['year','spend_spoc', 
        'total value of purchases_uk_finance',
       'Mastercard values_boe', 'Visa Europe values_boe',
       'Mastercard and Visa values_boe', 'Visa proportion_boe',
       'debit_spend_global', 'credit_spend_global', 'visa_total_spend_global',
       'total_spend_global', 'visa_marketshare_spend_global']]
spend = spend.copy()
# #replace 2024 spending with NA
spend['spend_spoc'] = np.where(spend['year']==2024, np.nan, spend['spend_spoc'])
spend['total value of purchases_uk_finance'] = np.where(spend['year']==2024, np.nan, spend['total value of purchases_uk_finance'])
#calculate marketshare
spend['uk_finance_marketshare'] = spend['spend_spoc'] / spend['total value of purchases_uk_finance'] *100
spend['global_marketshare'] = spend['spend_spoc'] / spend['total_spend_global'] *100
spend['boe_marketshare'] = spend['spend_spoc'] / spend['Mastercard and Visa values_boe'] *100
#copy used for getting 2019 marketshare
spend_copy = spend.copy()
#melt df for charts
spend = pd.melt(spend, id_vars='year',var_name='Data source', value_name='value')
spend = calc_utils.calculate_index(df = spend)

plot_utils.plot_total_cardholders(df = cardholders)

In [ ]:
#options of marketshare threshold
# marketshare_2019 = spend_copy.iloc[0]['visa_marketshare_spend_global']
# marketshare_2019 = spend_copy.iloc[0]['global_marketshare']
marketshare_2019 = spend_copy.iloc[0]['uk_finance_marketshare']
marketshare_2019

df = merged.copy() 
#remove 2024 due to incomplete data
df = df[df['year'] != 2024]
#index spoc data
df['idx_cardholders_spoc'] = df['cardholders_spoc'].transform(lambda x: (x / x.iloc[0] * 100))
df['idx_spend_spoc'] = df['spend_spoc'].transform(lambda x: (x / x.iloc[0] * 100))
# adjust visa spend to 2019 cardholders (assume same number of cardholders each year)
df['visa_adj_spend_spoc'] = (df['spend_spoc']/df['idx_cardholders_spoc'])*100
df['total_spoc'] = df['visa_adj_spend_spoc'] / marketshare_2019 *100
#rename columns
df = df.rename(columns={'total value of purchases_uk_finance': 'total_uk_finance'})
df = df.rename(columns={'visa_total_spend_global': 'visa_total_global','total_spend_global':'total_global'})
df = df.rename(columns={'Visa Europe values_boe': 'visa_total_boe', 'Mastercard and Visa values_boe': 'total_boe'})
#filter columns
df = df[['year', 'visa_adj_spend_spoc', 'total_spoc', 'visa_total_global', 'total_global', 'total_uk_finance', 'visa_total_boe', 'total_boe' ]]
#melt df
df = pd.melt(df, id_vars='year',var_name='Data source', value_name='value')
df = calc_utils.calculate_index(df = df)

# total_uk_finance: Total spend in the UK finance sector.
# visa_total_global: Total global spend for Visa.
# total_global: Total global spend (presumably for both Visa and Mastercard).
# visa_total_boe: Total spend for Visa in the Bank of England (BOE) area.
# total_boe: Total spend in the BOE area, for both Visa and Mastercard.

In [ ]:
import pandas as pd

# Assuming 'df' is your DataFrame after transformation and melting
# Filter the DataFrame for the relevant columns (those you want to display)
data_sources = ['total_uk_finance', 'visa_total_global', 'total_global', 'visa_total_boe', 'total_boe']

# Filter rows where 'Data source' is one of the specified sources
df_filtered = df[df['Data source'].isin(data_sources)]

# Pivot the DataFrame to get each data source as columns, showing values by year
df_pivot = df_filtered.pivot_table(index='year', columns='Data source', values='value', aggfunc='first')

# Display the result
print(df_pivot)


In [ ]:
# Given data for 2021, 2022, and 2023
spend_2023 = 1.34e+12
spend_2022 = 1.17e+12
spend_2021 = 1.10e+12

# Calculate yearly growth rates
growth_rate_2022_2023 = (spend_2023 / spend_2022) - 1
growth_rate_2021_2022 = (spend_2022 / spend_2021) - 1

# Average growth rate (you could also use other techniques to refine this)
avg_growth_rate = (growth_rate_2022_2023 + growth_rate_2021_2022) / 2

# Estimate 2020 and 2019 figures based on the average growth rate
spend_2020_est = spend_2021 / (1 + avg_growth_rate)  # Backtrack from 2021 to 2020
spend_2019_est = spend_2020_est / (1 + avg_growth_rate)  # Backtrack from 2020 to 2019

# Results
print(f"Estimated Spend for 2020: {spend_2020_est:.2e}")
print(f"Estimated Spend for 2019: {spend_2019_est:.2e}")


In [ ]:
import pandas as pd

# Sample data for df_pivot (replace this with your actual df_pivot)
df_pivot = pd.DataFrame({
    'total_uk_finance': [8.028740e+11, 8.016540e+11, 8.743800e+11, 9.654290e+11, 1.021071e+12],
    'visa_total_global': [8.120000e+11, 7.647000e+11, 7.960000e+11, 7.340000e+11, 6.945000e+11],
    'total_global': [9.694000e+11, 9.028000e+11, 9.903000e+11, 1.093500e+12, 1.171000e+12],
    'visa_total_boe': [8.624950e+11, 8.666880e+11, 9.055650e+11, 8.603050e+11, 8.219800e+11],
    'total_boe': [9.02e+11, 9.96e+11, 1.095000e+12, 1.165080e+12, 1.339185e+12]   # NaN values in 2019, 2020
}, index = [2019, 2020, 2021, 2022, 2023])

# Sample online_spend_ratio DataFrame
online_spend_ratio_data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'online_spend_ratio': [40.728857, 49.799012, 48.471550, 46.433003, 47.173528]
}
online_spend_ratio_df = pd.DataFrame(online_spend_ratio_data)

# Reset the index of df_pivot to move 'year' into a column
df_pivot = df_pivot.reset_index()

# Check the columns of df_pivot after reset_index to ensure 'year' is now a column
print("\nColumns in df_pivot after reset_index:")
print(df_pivot.columns)

# Ensure 'year' columns in both DataFrames are of the same type (integer)
df_pivot['year'] = df_pivot['index'].astype(int)
online_spend_ratio_df['year'] = online_spend_ratio_df['year'].astype(int)

# Now, we can proceed with the merge
df_merged = pd.merge(df_pivot, online_spend_ratio_df, on='year', how='left')

# Check for NaN values in total_boe and handle them (e.g., forward fill or set to 0)
df_merged['total_boe'] = df_merged['total_boe'].fillna(df_merged['total_boe'].mean())  # Filling NaN with mean or other strategies

# Now calculate the online spend for each data source
df_merged['online_total_uk_finance'] = df_merged['total_uk_finance'] * (df_merged['online_spend_ratio'] / 100)
df_merged['online_total_visa_global'] = df_merged['visa_total_global'] * (df_merged['online_spend_ratio'] / 100)
df_merged['online_total_global'] = df_merged['total_global'] * (df_merged['online_spend_ratio'] / 100)
df_merged['online_total_boe'] = df_merged['visa_total_boe'] * (df_merged['online_spend_ratio'] / 100)
df_merged['online_total_boe_total'] = df_merged['total_boe'] * (df_merged['online_spend_ratio'] / 100)

# Display the final DataFrame with the online totals
print("\nFinal DataFrame with online totals:")
print(df_merged[['year', 'online_spend_ratio', 'online_total_uk_finance', 'online_total_visa_global', 
                 'online_total_global', 'online_total_boe', 'online_total_boe_total']])


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df_merged is already prepared as per the previous steps

# Define the data sources for plotting
data_sources = ['total_uk_finance', 'visa_total_global', 'total_global', 'visa_total_boe', 'total_boe']
online_data_sources = ['online_total_uk_finance', 'online_total_visa_global', 'online_total_global', 'online_total_boe', 'online_total_boe_total']

# Create a bar chart for each data source

# Create the bar chart for total values
bar_traces = []
for source in data_sources:
    bar_traces.append(go.Bar(
        x=df_merged['year'],
        y=df_merged[source],
        name=source
    ))

# Create the bar chart for online values
bar_online_traces = []
for source in online_data_sources:
    bar_online_traces.append(go.Bar(
        x=df_merged['year'],
        y=df_merged[source],
        name=source
    ))

# Plotting the Bar Charts for Total Spend and Online Spend
fig_bar = go.Figure(bar_traces + bar_online_traces)

fig_bar.update_layout(
    title="Total Spend vs Online Spend by Data Source",
    barmode='group',
    xaxis_title="Year",
    yaxis_title="Amount",
    template="plotly_dark"
)

# Display Bar Chart
fig_bar.show()


# Now, let's create a Line Chart to show trends over time
line_traces = []
for source in data_sources:
    line_traces.append(go.Scatter(
        x=df_merged['year'],
        y=df_merged[source],
        mode='lines+markers',
        name=source
    ))

# Line chart for online data sources
line_online_traces = []
for source in online_data_sources:
    line_online_traces.append(go.Scatter(
        x=df_merged['year'],
        y=df_merged[source],
        mode='lines+markers',
        name=source
    ))

# Plotting the Line Charts for Total Spend and Online Spend
fig_line = go.Figure(line_traces + line_online_traces)

fig_line.update_layout(
    title="Total Spend vs Online Spend Trend by Data Source",
    xaxis_title="Year",
    yaxis_title="Amount",
    template="plotly_dark"
)

# Display Line Chart
fig_line.show()


In [ ]:
import plotly.graph_objects as go
import pandas as pd

# Assuming df_merged contains the data you want to visualize

# Define the data sources for plotting
data_sources = ['total_uk_finance', 'visa_total_global', 'total_global', 'visa_total_boe', 'total_boe']
online_data_sources = ['online_total_uk_finance', 'online_total_visa_global', 'online_total_global', 'online_total_boe', 'online_total_boe_total']

# Create a bar chart for total spend values
bar_traces_total = []
for source in data_sources:
    bar_traces_total.append(go.Bar(
        x=df_merged['year'],
        y=df_merged[source],
        name=source
    ))

# Plotting the Bar Chart for Total Spend
fig_bar_total = go.Figure(bar_traces_total)

fig_bar_total.update_layout(
    title="Total Spend by Data Source",
    barmode='group',  # Group bars together
    xaxis_title="Year",
    yaxis_title="Amount in Trillions GBP",
    template="plotly_dark"
)

# Display Bar Chart for Total Spend
fig_bar_total.show()


# Now, let's create the Line Chart for Total Spend
line_traces_total = []
for source in data_sources:
    line_traces_total.append(go.Scatter(
        x=df_merged['year'],
        y=df_merged[source],
        mode='lines+markers',  # Line chart with markers
        name=source
    ))

# Line Chart for Total Spend
fig_line_total = go.Figure(line_traces_total)

fig_line_total.update_layout(
    title="Total Spend Trend by Data Source",
    xaxis_title="Year",
    yaxis_title="Amount in Trillions GBP",
    template="plotly_dark"
)

# Display Line Chart for Total Spend
fig_line_total.show()


# Create a bar chart for online spend values
bar_traces_online = []
for source in online_data_sources:
    bar_traces_online.append(go.Bar(
        x=df_merged['year'],
        y=df_merged[source],
        name=source
    ))

# Plotting the Bar Chart for Online Spend
fig_bar_online = go.Figure(bar_traces_online)

fig_bar_online.update_layout(
    title="Online Spend by Data Source",
    barmode='group',  # Group bars together
    xaxis_title="Year",
    yaxis_title="Amount in Billions GBP",
    template="plotly_dark"
)

# Display Bar Chart for Online Spend
fig_bar_online.show()


# Now, let's create the Line Chart for Online Spend
line_traces_online = []
for source in online_data_sources:
    line_traces_online.append(go.Scatter(
        x=df_merged['year'],
        y=df_merged[source],
        mode='lines+markers',  # Line chart with markers
        name=source
    ))

# Line Chart for Online Spend
fig_line_online = go.Figure(line_traces_online)

fig_line_online.update_layout(
    title="Online Spend Trend by Data Source",
    xaxis_title="Year",
    yaxis_title="Amount in Billions GBP",
    template="plotly_dark"
)

# Display Line Chart for Online Spend
fig_line_online.show()


In [ ]:
#UK Abroad Online Ratio

year  online_spend_ratio (With B2B)
0  2019           71.307774
1  2020           83.569366
2  2021           82.616680
3  2022           63.693530
4  2023           61.288192
5  2024           59.669720

Without B2B Adj2
66.76842075
78.53540897
75.46094019
56.06727668
54.12817165

Household Abroad online spend ratio
4.87421
4.94802
4.37668
3.25206
3.35606


#UK Abroad Total
   year         spend
0  2019  4.162683e+10
1  2020  3.659158e+10
2  2021  3.586756e+10
3  2022  3.485071e+10
4  2023  3.577281e+10
5  2024  2.890089e+10

#UK Abroad Spending Online Total (Visa)
   year  online_spend
0  2019  2.968316e+10
1  2020  3.057935e+10
2  2021  2.963259e+10
3  2022  2.219765e+10
4  2023  2.192451e+10
5  2024  1.724508e+10


#UK Total Domestic

Data source    spend_spoc  total value of purchases_uk_finance  \
year                                                             
2019         5.155348e+11                         8.028740e+11   
2020         5.218997e+11                         8.016540e+11   
2021         5.509458e+11                         8.743800e+11   
2022         5.316535e+11                         9.654290e+11   
2023         5.108432e+11                         1.021071e+12   

# Formula - {[total/( total+ spend_spoc)*100]* total value of purchases_uk_finance }* online_spend_ratio/100 = uk_abroad_total_for_uk_finance



In [ ]:
import pandas as pd

# Data for the years 2019 to 2023
years = [2019, 2020, 2021, 2022, 2023]
total_abroad = [4.162683e+10, 3.659158e+10, 3.586756e+10, 3.485071e+10, 3.577281e+10]
spend_spoc = [5.155348e+11, 5.218997e+11, 5.509458e+11, 5.316535e+11, 5.108432e+11]
total_value_uk_finance = [8.028740e+11, 8.016540e+11, 8.743800e+11, 9.654290e+11, 1.021071e+12]
online_spend_ratio = [71.30776,
83.569362,
82.616688,
63.693537,
61.288196
]

# Create a DataFrame to hold the data
data = {
    'year': years,
    'total_abroad': total_abroad,
    'spend_spoc': spend_spoc,
    'total_value_uk_finance': total_value_uk_finance,
    'online_spend_ratio': online_spend_ratio
}

df = pd.DataFrame(data)

# Calculate the UK Abroad Total for UK Finance for each year using the formula
df['uk_abroad_total_for_uk_finance'] = (
    ((df['total_abroad'] / (df['total_abroad'] + df['spend_spoc']))) * df['total_value_uk_finance'] 
    * (df['online_spend_ratio'] / 100)
)

# Display the resulting DataFrame
print(df[['year', 'uk_abroad_total_for_uk_finance']])



In [ ]:
pip install plotly


In [ ]:
   #Household abroad online spend ratio
     year  Household_Abroad_Online_Spend_Ratio
0  20.19                             4.846420
1  20.20                             4.963785
2  20.21                             4.370532
3  20.22                             3.240852
4  20.23                             3.343671
5  20.24                             3.528571

#Total Spending Abroad Visa
41626830000
36591580000
35867560000
34850710000
35772810000

#total_abroad = [4.162683e+10, 3.659158e+10, 3.586756e+10, 3.485071e+10, 3.577281e+10]
#spend_spoc = [5.155348e+11, 5.218997e+11, 5.509458e+11, 5.316535e+11, 5.108432e+11]


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Data for the years 2019 to 2023
years = [2019, 2020, 2021, 2022, 2023]
total_value_uk_finance = [8.028740e+11, 8.016540e+11, 8.743800e+11, 9.654290e+11, 1.021071e+12]
online_spend_ratio = [4.846420, 4.963785, 4.370532, 3.240852, 3.343671]

# Create a DataFrame to hold the data
data = {
    'year': years,
    'total_value_uk_finance': total_value_uk_finance,
    'online_spend_ratio': online_spend_ratio
}

df = pd.DataFrame(data)

# Calculate the UK Abroad Total for UK Finance for each year using the formula
df['uk_abroad_total_for_uk_finance'] = (
    (df['total_value_uk_finance'] 
    * df['online_spend_ratio'] / 100)
)

# Create Bar chart for UK Abroad Total for UK Finance
bar_chart = go.Bar(
    x=df['year'],
    y=df['uk_abroad_total_for_uk_finance'],
    name='UK Abroad Total for UK Finance',
    marker=dict(color='blue')
)

# Create the Bar chart layout and show it
fig_bar = go.Figure(data=[bar_chart])
fig_bar.update_layout(
    title='UK Abroad Online Total for UK Finance (2019-2023)',
    xaxis_title='Year',
    yaxis_title='Amount in Billions (£)',
    template='plotly_dark'
)

# Display the bar chart
fig_bar.show()

# Create Line chart for Online Spend Ratio
line_chart = go.Scatter(
    x=df['year'],
    y=df['online_spend_ratio'],
    mode='lines+markers',
    name='Online Spend Ratio (%)',
    line=dict(color='red', width=2),
    marker=dict(color='red', size=8)
)

# Create the Line chart layout and show it
fig_line = go.Figure(data=[line_chart])
fig_line.update_layout(
    title='Online Spend Ratio (2019-2023)',
    xaxis_title='Year',
    yaxis_title='Online Spend Ratio (%)',
    template='plotly_dark'
)

# Display the line chart
fig_line.show()


In [ ]:
# Display the table with the UK Abroad Total for UK Finance
df_table = df[['year', 'uk_abroad_total_for_uk_finance']]

# Show the table
print(df_table)

In [ ]:
#Need to Change Values for total_boe

import pandas as pd

# Data for the years 2019 to 2023
years = [2019, 2020, 2021, 2022, 2023]
total_abroad = [4.162683e+10, 3.659158e+10, 3.586756e+10, 3.485071e+10, 3.577281e+10]
spend_spoc = [5.155348e+11, 5.218997e+11, 5.509458e+11, 5.316535e+11, 5.108432e+11]
visa_total_boe = [8.028740e+11, 8.016540e+11, 8.743800e+11, 9.654290e+11, 1.021071e+12]
online_spend_ratio = [71.307774, 83.569366, 82.616680, 63.693530, 61.288192]

# Create a DataFrame to hold the data
data = {
    'year': years,
    'total_abroad': total_abroad,
    'spend_spoc': spend_spoc,
    'visa_total_boe': visa_total_boe,
    'online_spend_ratio': online_spend_ratio
}

df = pd.DataFrame(data)

# Calculate the UK Abroad Total for UK Finance for each year using the formula
df['uk_abroad_total_for_boe_visa'] = (
    ((df['total_abroad'] / (df['total_abroad'] + df['spend_spoc']))) * df['visa_total_boe'] 
    * (df['online_spend_ratio'] / 100)
)

# Display the resulting DataFrame
print(df[['year', 'uk_abroad_total_for_boe_visa']])



In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Data for the years 2019 to 2023
years = [2019, 2020, 2021, 2022, 2023]
total_abroad = [8.531143e+10, 8.802151e+10, 8.518794e+10, 6.329587e+10, 6.289442e+10]
spend_spoc = [5.155348e+11, 5.218997e+11, 5.509458e+11, 5.316535e+11, 5.108432e+11]
total_value_uk_finance = [8.028740e+11, 8.016540e+11, 8.743800e+11, 9.654290e+11, 1.021071e+12]
online_spend_ratio = [71.307774, 83.569366, 82.616680, 63.693530, 61.288192]

# Create a DataFrame to hold the data
data = {
    'year': years,
    'total_abroad': total_abroad,
    'spend_spoc': spend_spoc,
    'total_value_uk_finance': total_value_uk_finance,
    'online_spend_ratio': online_spend_ratio
}

df = pd.DataFrame(data)

# Calculate the UK Abroad Total for UK Finance for each year using the formula
df['uk_abroad_total_for_uk_finance_adj'] = (
    ((df['total_abroad'] / (df['total_abroad'] + df['spend_spoc'])) * 100) * df['total_value_uk_finance'] 
    * (df['online_spend_ratio'] / 100)
)

# Create Bar chart for UK Abroad Total for UK Finance
bar_chart = go.Bar(
    x=df['year'],
    y=df['uk_abroad_total_for_uk_finance_adj'],
    name='UK Abroad Total for UK Finance_adj',
    marker=dict(color='blue')
)

# Create the Bar chart layout and show it
fig_bar = go.Figure(data=[bar_chart])
fig_bar.update_layout(
    title='UK Abroad Online Total for UK Finance Adj (2019-2023)',
    xaxis_title='Year',
    yaxis_title='Amount (£)',
    template='plotly_dark'
)

# Display the bar chart
fig_bar.show()

# Create Line chart for Online Spend Ratio
line_chart = go.Scatter(
    x=df['year'],
    y=df['online_spend_ratio'],
    mode='lines+markers',
    name='Online Spend Ratio (%)',
    line=dict(color='red', width=2),
    marker=dict(color='red', size=8)
)

# Create the Line chart layout and show it
fig_line = go.Figure(data=[line_chart])
fig_line.update_layout(
    title='Online Spend Ratio (2019-2023)',
    xaxis_title='Year',
    yaxis_title='Online Spend Ratio (%)',
    template='plotly_dark'
)

# Display the line chart
fig_line.show()


In [ ]:
import pandas as pd
import plotly.graph_objects as go

# Data for the years 2019 to 2023
years = [2019, 2020, 2021, 2022, 2023]
total_abroad = [8.33E+10, 7.32E+10, 7.17E+10, 6.97E+10, 7.15E+10]
spend_spoc = [5.155348e+11, 5.218997e+11, 5.509458e+11, 5.316535e+11, 5.108432e+11]
total_value_uk_finance = [8.028740e+11, 8.016540e+11, 8.743800e+11, 9.654290e+11, 1.021071e+12]
online_spend_ratio = [71.307774, 83.569366, 82.616680, 63.693530, 61.288192]

# Create a DataFrame to hold the data
data = {
    'year': years,
    'total_abroad': total_abroad,
    'spend_spoc': spend_spoc,
    'total_value_uk_finance': total_value_uk_finance,
    'online_spend_ratio': online_spend_ratio
}

df = pd.DataFrame(data)

# Calculate the UK Abroad Total for UK Finance for each year using the formula
df['uk_abroad_total_for_uk_finance_adj1'] = (
    ((df['total_abroad'] / (df['total_abroad'] + df['spend_spoc']))) * df['total_value_uk_finance'] 
    * (df['online_spend_ratio'] / 100)
)

# Create Bar chart for UK Abroad Total for UK Finance
bar_chart = go.Bar(
    x=df['year'],
    y=df['uk_abroad_total_for_uk_finance_adj1'],
    name='UK Abroad Total for UK Finance_adj1',
    marker=dict(color='blue')
)

# Create the Bar chart layout and show it
fig_bar = go.Figure(data=[bar_chart])
fig_bar.update_layout(
    title='UK Abroad Online Total for UK Finance Adj1 (2019-2023)',
    xaxis_title='Year',
    yaxis_title='Amount (£)',
    template='plotly_dark'
)

# Display the bar chart
fig_bar.show()

# Create Line chart for Online Spend Ratio
line_chart = go.Scatter(
    x=df['year'],
    y=df['online_spend_ratio'],
    mode='lines+markers',
    name='Online Spend Ratio (%)',
    line=dict(color='red', width=2),
    marker=dict(color='red', size=8)
)

# Create the Line chart layout and show it
fig_line = go.Figure(data=[line_chart])
fig_line.update_layout(
    title='Online Spend Ratio (2019-2023)',
    xaxis_title='Year',
    yaxis_title='Online Spend Ratio (%)',
    template='plotly_dark'
)

# Display the line chart
fig_line.show()


In [ ]:
# Display the table with the UK Abroad Total for UK Finance
df_table = df[['year', 'uk_abroad_total_for_uk_finance_adj1']]

# Show the table
print(df_table)

In [ ]:
import plotly.express as px
import pandas as pd

# Data for UK Finance, Global Data, and BoE
data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK Finance': [38.15, 38.97, 37.37, 30.65, 33.43],
    'Global Report': [45.14, 42.95, 41.42, 34.03, 37.62],
    'BoE': [42.85, 48.40, 46.82, 37.00, 43.89]
}

# Create dataframe
df = pd.DataFrame(data)

# Create the line chart
fig_line = px.line(df, 
                   x='year', 
                   y=['UK Finance', 'Global Report', 'BoE'], 
                   title='UK Abroad Online Spending UK Finance, Global Report, and BoE 2019 - 2023',
                   labels={'year': 'Year', 'UK Finance': 'UK Finance (£Bn)', 'Global Report': 'Global Report (£Bn)', 'BoE': 'BoE (£Bn)'},
                   markers=True)  # Add markers at each data point

# Customize the line chart appearance
fig_line.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Value (£Bn)'))

# Show the line chart
fig_line.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Data for each category
data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'Total Card Spend (Visa)': [557.16, 558.49, 586.81, 566.50, 546.62],
    'UK Finance Total Spend': [803.00, 802.00, 874.00, 965.00, 1020.00],
    'Global Report Total Spend': [950.23, 883.80, 968.75, 1071.48, 1147.85],
    'BOE Total Spend': [902.00, 996.00, 1095.00, 1165.08, 1339.19]
}

# Create dataframe
df = pd.DataFrame(data)

# Create the line chart
fig_line = px.line(df, 
                   x='year', 
                   y=['Total Card Spend (Visa)', 'UK Finance Total Spend', 'Global Report Total Spend', 'BOE Total Spend'], 
                   title='UK Total Card Spend Visa, UK Finance, Global Report, BOE',
                   labels={'year': 'Year', 
                           'Total Card Spend (Visa)': 'Total Card Spend (Visa) (£Bn)', 
                           'UK Finance Total Spend': 'UK Finance Total Spend (£Bn)', 
                           'Global Report Total Spend': 'Global Report Total Spend (£Bn)', 
                           'BOE Total Spend': 'BOE Total Spend (£Bn)'},
                   markers=True)  # Add markers at each data point

# Customize the line chart appearance
fig_line.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Spend (£Bn)'))

# Show the line chart
fig_line.show()

In [ ]:
import plotly.express as px
import pandas as pd

# Data for UK Finance, Global Report, and BoE
data = {
    'year': [2019, 2020, 2021, 2022, 2023],
    'UK Finance': [3.270014e+11, 3.992158e+11, 4.238255e+11, 4.482777e+11, 4.816752e+11],
    'Global Report': [3.948255e+11, 4.495855e+11, 4.800138e+11, 5.077449e+11, 5.524020e+11],
    'BoE': [3.673743e+11, 4.959982e+11, 5.307635e+11, 5.409816e+11, 6.317408e+11]
}

# Create dataframe
df = pd.DataFrame(data)

# Create the line chart
fig_line = px.line(df, 
                   x='year', 
                   y=['UK Finance', 'Global Report', 'BoE'], 
                   title='UK Domestic Online Spending: UK Finance, Global Report, and BoE 2019 - 2023',
                   labels={'year': 'Year', 'UK Finance': 'UK Finance (£)', 'Global Report': 'Global Report (£)', 'BoE': 'BoE (£)'},
                   markers=True)  # Add markers at each data point

# Customize the line chart appearance
fig_line.update_traces(line=dict(width=3), marker=dict(size=7))
fig_line.update_layout(xaxis=dict(tickmode='linear'), yaxis=dict(title='Value (£)'))

# Show the line chart
fig_line.show()